1. Как устроен современный веб. Характеристика клиент-серверной архитектуры, основные протоколы обмена данными и форматы их хранения.
2. Загрузка данных при помощи модуля request. Загрузка данных с обычной интернет-страницы, загрузка динамически подгружаемого контента, загрузка данных с защищённых страниц, подключение к API (на примере vk.com и meduza.io).
3. Структура веб-страницы. Извлечение необходимых данных из HTML при помощи библиотеки BeautifulSoup.
4. Сохранение данных в файл.
5. Практическое задание — «Писец Питерской Вышки». Соберите информацию с сайта ВШЭ-СПб и определите сотрудника с наибольшим количеством публикаций.
6. Бонус: многократное ускорение сбора данных при помощи асинхронных запросов.

# Как устроен веб?

## Клиент-серверная архитектура

Компьютеры, подключенные к сети называются клиентами и серверами. Схема обмена информацией выглядит приблизительно так:
![image.png](https://mdn.mozillademos.org/files/8973/Client-server.jpg)

Клиенты являются обычными пользователями, подключенными к Интернету посредством устройств и программного обеспечения, доступного на этих устройствах (как правило, браузер, например, Firefox или Chrome).

Серверы - компьютеры, которые хранят веб-страницы, сайты или приложения. Когда клиентское устройство пытается получить доступ к веб-странице, копия страницы загружается с сервера на клиентский компьютер для отображения в браузере пользователя.


## HTTP как основной протокол передачи данных

Связь между клиентом и сервером осуществляется по определённому протоколу, обычно это HTTP (HyperText Transfer Protocol). Каждое HTTP-сообщение состоит из трёх частей, которые передаются в указанном порядке:

* Стартовая строка (Starting line) — определяет тип сообщения;
* Заголовки (Headers) — характеризуют тело сообщения, параметры передачи и прочие сведения;
* Тело сообщения (Message Body) — непосредственно данные сообщения. Обязательно должно отделяться от заголовков пустой строкой.

Тело сообщения может отсутствовать, но стартовая строка и заголовок являются обязательными элементами.


### Стартовая строка

Стартовые строки различаются для запроса и ответа. 

**Строка запроса** выглядит так: **`Метод URI HTTP/Версия`**. 

Самые часто используемые методы это `GET` и `POST` (ещё существую `DELETE`, `PUT`, `HEAD` и другие).

`GET` используется для запроса содержимого указанного ресурса, `POST` применяется для передачи пользовательских данных заданному ресурсу. 

Структура URI: `<схема>:[//[<логин>:<пароль>@]<хост>[:<порт>]][/<URL‐путь>][?<параметры>][#<якорь>]`.
![](https://habrastorage.org/files/373/2b3/3fd/3732b33fd43043049c18e3c108bc9d1a.jpg)

**Cтрока ответа** сервера имеет следующий формат: **`HTTP/Версия КодСостояния Пояснение`**.

Код состояния является частью первой строки ответа сервера. Он представляет собой целое число из трёх цифр. Первая цифра указывает на класс состояния. За кодом ответа обычно следует отделённая пробелом поясняющая фраза на английском языке, которая разъясняет человеку причину именно такого ответа. [Список кодов состояния на wiki](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP).

### [Заголовки](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BA%D0%B8_HTTP)

Заголовки HTTP (англ. HTTP Headers) — это строки в HTTP-сообщении, содержащие разделённую двоеточием пару имя-значение. 

### Тело страницы

HTML, JSON


# Загрузка данных при помощи модуля request

Библиотека requests — это обёртка над другой, более низкоуровневой библиотекой urllib3, упрощающая доступ ко многим функциям.

В requests имеется:

* Множество методов http аутентификации
* Сессии с куками
* Полноценная поддержка SSL
* Различные методы .json(), которые вернут данные в нужном формате
* Проксирование
* Грамотная и логичная работа с исключениями

## Загрузка данных с обычной интернет-страницы

Вот как выглядит HTTP-запрос методом `GET` с помощью requests:

In [2]:
import requests
r = requests.get("https://www.hse.ru/")
print(f"Код состояния: {r.status_code}.")
print(f"Заголовки: {r.headers['content-type']}.")

Код состояния: 200.
Заголовки: text/html; charset=utf-8.


Получить HTML можно с помощью свойства `text`:

In [4]:
r.text[:500]

'<!DOCTYPE html>\n<html lang="ru">\n\n\n\n\n\n\n\n\n\n\n<head>\n\t<title>Национальный исследовательский университет Высшая школа экономики</title>\n\t<meta charset="utf-8" />\n\t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n\t<meta name="yandex-verification" content="25e3420f8bfc397e" />\n\t<meta name="theme-color" content="#1658DA"/>\n\t<meta name="sputnik-verification" content="SB2V6wrEtxYZNqgs" />\n\t<link rel="manifest" href="https://www.'

## Загрузка динамически подгружаемого контента

В качестве примера рассмотрим сайт meduza.io. Зайдиде на него и посмотрите через Chrome DevTools, как в нём динамечески подгружается контент. Определим url, на который делаются запросы.

In [5]:
meduza_url = "https://meduza.io/api/v3/search?chrono=news&page={}&per_page=24&locale=ru"

In [7]:
requests.get(meduza_url.format(0)).text[:500]

'{"has_next":true,"documents":{"slides/ostavte-nas-v-pokoe-so-svoey-kulturoy-dagestantsy-i-habib-nurmagomedov-possorilis-s-timati-i-leyblom-black-star-a-pomirit-ih-pytaetsya-kadyrov":{"with_banners":true,"version":5,"url":"slides/ostavte-nas-v-pokoe-so-svoey-kulturoy-dagestantsy-i-habib-nurmagomedov-possorilis-s-timati-i-leyblom-black-star-a-pomirit-ih-pytaetsya-kadyrov","title":"«Оставьте нас в\xa0покое со\xa0своей культурой». Дагестанцы и\xa0Хабиб Нурмагомедов поссорились с\xa0Тимати и\xa0лейблом Black Star\xa0—'

In [8]:
requests.get(meduza_url.format(0)).json().keys()

dict_keys(['has_next', 'documents', 'collection', '_count'])

In [9]:
for url, doc in requests.get(meduza_url.format(0)).json()["documents"].items():
    print("Заголовок: " + doc["title"])

Заголовок: «Оставьте нас в покое со своей культурой». Дагестанцы и Хабиб Нурмагомедов поссорились с Тимати и лейблом Black Star — а помирить их пытается Кадыров
Заголовок: «А вы точно не сотрудники спецслужб? — Так точно!»: пользователи соцсетей высмеивают интервью Петрова и Боширова
Заголовок: Усэйн Болт выиграл забег в невесомости. Теперь он самый быстрый человек на земле и в космосе
Заголовок: Магические существа и тайны города Тролльберг в первом трейлере «Хильды» — нового мультсериала Netflix
Заголовок: «Вся эта ситуация — фантастическое совпадение». Краткий пересказ интервью Александра Петрова и Руслана Боширова о Солсбери
Заголовок: Законопроект об уголовной ответственности за увольнение работников предпенсионного возраста принят Госдумой в первом чтении
Заголовок: Врачи заподозрили у Петра Верзилова отравление лекарствами. Близкие говорят, что он ничего не принимал
Заголовок: Ватикан начал расследование мошенничества в Сикстинской капелле
Заголовок: В Китае водитель внедорожник

In [10]:
for page in range(10):
    for url, doc in requests.get(meduza_url.format(page)).json()["documents"].items():
        print("Заголовок: " + doc["title"])

Заголовок: «Оставьте нас в покое со своей культурой». Дагестанцы и Хабиб Нурмагомедов поссорились с Тимати и лейблом Black Star — а помирить их пытается Кадыров
Заголовок: «А вы точно не сотрудники спецслужб? — Так точно!»: пользователи соцсетей высмеивают интервью Петрова и Боширова
Заголовок: Усэйн Болт выиграл забег в невесомости. Теперь он самый быстрый человек на земле и в космосе
Заголовок: Магические существа и тайны города Тролльберг в первом трейлере «Хильды» — нового мультсериала Netflix
Заголовок: «Вся эта ситуация — фантастическое совпадение». Краткий пересказ интервью Александра Петрова и Руслана Боширова о Солсбери
Заголовок: Законопроект об уголовной ответственности за увольнение работников предпенсионного возраста принят Госдумой в первом чтении
Заголовок: Врачи заподозрили у Петра Верзилова отравление лекарствами. Близкие говорят, что он ничего не принимал
Заголовок: Ватикан начал расследование мошенничества в Сикстинской капелле
Заголовок: В Китае водитель внедорожник

Заголовок: «Никто как следует не давал по заднице, оппозиционная моська, отбивную сделаю». Как глава Росгвардии говорит про Навального
Заголовок: Не катается на велосипеде, не читает новости, не говнюк. 14 фактов о Марке Цукерберге из большого материала The New Yorker
Заголовок: Росгвардия (по-разному!) отвечает на расследование Навального. В трех частях
Заголовок: «Сейчас ты мне ответишь, каналья!»: в соцсетях шутят про дуэль Золотова и Навального
Заголовок: Глава Росгвардии вызвал Навального на дуэль: «Если съедете — будете слизняком». Максимально коротко
Заголовок: «Я обещаю сделать из вас хорошую сочную отбивную». Глава Росгвардии вызвал Навального на дуэль
Заголовок: ВЦИОМ: россияне стали меньше бояться терактов
Заголовок: В работе «Яндекса» произошел сбой
Заголовок: В ФБК напомнили главе Росгвардии, что Навальный не может принять вызов на дуэль из-за ареста
Заголовок: В Буденновске задержали студента, обливавшего кислотой школьниц
Заголовок: Следователя по делу об убийстве Немцов

Заголовок: Том Йорк, Фейс, Земфира, Маккартни, Idles — а также модные ребята со взглядом в пустоту
Заголовок: Колючая проволока на акции против пенсионной реформы в Петербурге. Фотография
Заголовок: Полтавченко предложил ставить предупреждающие билборды рядом с клубами, где продают наркотики. Хм-м-м-м
Заголовок: В Москве задержан водитель, который поссорился с посетителями кафе и въехал в них
Заголовок: Титановый завод в Армянске приостановил работу
Заголовок: Прокуратура США признала, что Мария Бутина не предлагала секс в обмен на должность
Заголовок: Президент Молдавии попал в аварию
Заголовок: Премьер Абхазии погиб в автокатастрофе
Заголовок: На акции сторонников Навального в Петербурге задержали 80 человек
Заголовок: Валентина Матвиенко встретилась с Ким Чен Ыном
Заголовок: СК возбудил уголовное дело об избиении заключенных в керченской ИК-2
Заголовок: РПЦ пригрозила разорвать отношения с Константинополем, если украинская церковь получит независимость
Заголовок: Российская фигурист

# Загрузка информации с защищённых сайтов

Загрузим информацию с сайта мониторинга трудоустройства выпускников: http://vo.graduate.edu.ru/

In [11]:
par_dict = {"id":45,"page":1,"params":{}}

In [22]:
headers = {
     "Accept": "application/json, text/javascript, */*; q=0.01",
     "Accept-Encoding": "gzip, deflate",
     "Accept-Language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,la;q=0.6",
     "Connection": "keep-alive",
     "Content-Length": "100",
     "Content-Type": "application/json; charset=UTF-8",
     "Cookie": "_ym_uid=15260890961045257376; _ym_isad=1; _ym_visorc_31062401=w; _vagrant_session=V2Qwb3V0aWc4K3NDVW1KVkdxQ0xzZjEyL3JIM2JuSlBGeFJyTEJhUHVXZkZEWnArdUJ6eEVXRFJBcmhrQVh0dWtpVG9iQ0g0UDJXNmtIR0lYUkhJaDhEMkVscUdvSzZhOUFJSlVDSUlqTTlzdXJEY0dpY1Jsa1Q5SzRSb01VdDRDWGVuaGtzSStaazYyYmdnOWFxcWRvSkJ2RFhxM0hadHdyT3hUNEgrY0hFPS0tUTNrUmY2eUx0bXVQdDlYbTkxZzFhQT09--346f51dc5bbd378a982946b4dc5b95d5b5e3038a",
     "Host": "vo.graduate.edu.ru",
     "Origin": "http://vo.graduate.edu.ru",
     "Referer": "http://vo.graduate.edu.ru/registry",
     "Save-Data": "on",
     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36",
     "X-CSRF-Token": "ZxpQgT3/JsEF/evo5FFLH2niFgeDSrdl8BLQ798xzHiG3umyqLy/Yt61aMwx9jd92/2hitrsaSGN5siwVLrGRQ==",
     "X-Requested-With": "XMLHttpRequest"
}

In [18]:
par_json = json.dumps(par_dict)

In [19]:
r = requests.post("http://vo.graduate.edu.ru/graphs/getGraph", data=par_json)

In [20]:
r

<Response [422]>

In [23]:
r = requests.post("http://vo.graduate.edu.ru/graphs/getGraph", data=par_json, headers=headers)
r

<Response [200]>

# HTML и его парсинг с помощью BeautifulSoup

HTML — теговый язык разметки документов. Любой документ на языке HTML представляет собой набор элементов, причём начало и конец каждого элемента обозначается специальными пометками — тегами. Элементы могут быть пустыми, то есть не содержащими никакого текста и других данных (например, тег перевода строки `<br>`). В этом случае обычно не указывается закрывающий тег. Кроме того, элементы могут иметь атрибуты, определяющие какие-либо их свойства (например, размер шрифта для тега <font>). Атрибуты указываются в открывающем теге. Вот примеры фрагментов HTML-документа:

даст следующее:

> <strong>Текст между двумя тегами — открывающим и закрывающим.</strong>

> <a href="http://www.example.com">Здесь элемент содержит атрибут href, то есть гиперссылку.</a>

> А вот пример пустого элемента: <br> и какой-то текст

Регистр, в котором набрано имя элемента и имена атрибутов, в HTML значения не имеет (в отличие от XHTML). Элементы могут быть вложенными. 

Для парсинга HTML существуют разные библиотеки, но чаще всего используется [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). Для тех, кто занком с jQuery более удобным вариантом может быть библиотека [pyquery](https://pythonhosted.org/pyquery/).

In [ ]:
!pip3 install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup

BS поддерживает разные парсеры html: https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser

In [32]:
soup = BeautifulSoup("какой-то текст <b class='class_name', id='id_attr'>Полужирный текст</b> <i>И ещё немножко</i>", "lxml")
soup

<html><body><p>какой-то текст <b class="class_name" id="id_attr">Полужирный текст</b> <i>И ещё немножко</i></p></body></html>

In [33]:
btag = soup.b
btag

<b class="class_name" id="id_attr">Полужирный текст</b>

In [34]:
dir(btag)

['HTML_FORMATTERS',
 'XML_FORMATTERS',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_attr_value_as_string',
 '_attribute_checker',
 '_find_all',
 '_find_one',
 '_formatter_for_name',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_select_debug',
 '_selector_combinators',
 '_should_pretty_print',
 '_tag_name_matches_and',
 'append',
 'attribselect_re',
 'attrs',
 'can_be_empty_element',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decomp

In [35]:
btag.name

'b'

In [36]:
btag.name = "span"
btag

<span class="class_name" id="id_attr">Полужирный текст</span>

In [37]:
btag["class"]

['class_name']

In [38]:
btag.attrs

{'class': ['class_name'], 'id': 'id_attr'}

In [39]:
btag["id"] = "some_id"
btag

<span class="class_name" id="some_id">Полужирный текст</span>

In [40]:
btag.string

'Полужирный текст'

In [41]:
btag.string.replace_with("Новый текст")
btag

<span class="class_name" id="some_id">Новый текст</span>

In [42]:
soup.get_text()

'какой-то текст Новый текст И ещё немножко'

In [5]:
sibling_soup = BeautifulSoup("<a><b class='cl'>text1</b><b class='cl'>text2</b></b></a>", "lxml")
print(sibling_soup.prettify())

<html>
 <body>
  <a>
   <b class="cl">
    text1
   </b>
   <b class="cl">
    text2
   </b>
  </a>
 </body>
</html>


In [7]:
sibling_soup.find("b", attrs={"class": "cl"})
sibling_soup.find("b", class_="cl")

<b class="cl">text1</b>

In [9]:
all_b = sibling_soup.find_all("b", class_="cl")
all_b

[<b class="cl">text1</b>, <b class="cl">text2</b>]

In [10]:
for b in all_b:
    print(b.get_text())

text1
text2


# Самостоятельная работа

Скачайте профили всех сотрудников питерской вышки и определите, у кого из них больше всего публикаций. Переменные в помощь:

In [14]:
CYRILLIC_ALPHBET = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
HSE_URL = "https://www.hse.ru/org/persons/?ltr={letter};udept={campus_id}"

# Сложности

Не всегда получить текст страницы бывает так просто, поскольку в современные веб-приложения загружают контент динамически, а URL при этом не изменяется (хотя должен бы). Для примера посмотрите на сайты https://www.1tv.ru/news и https://gorod55.ru/news. В таком случае при помощи Инструментов разработчика придётся отслеживать, какие запросы делает страница, и искать среди них те, которые возвращают нужную информацю.

# Асинхронные запросы

Синхронные операции — операции, при которых мы получаем результат в результате блокирования потока выполнения.

Асинхронные операции — операции, при которых мы просим совершить некоторую операцию и можем каким-либо образом отслеживать процесс/результат её выполнения. Когда она будет выполнена — неизвестно, но мы можем продолжить заниматься другими делами.

Написание асинхронного кода [не самая простая задача для разработчика](http://lucumr.pocoo.org/2016/10/30/i-dont-understand-asyncio/), но именно асинхронность может повысить производительность вашей программы в несколько десятков раз.

Ключевое слово [**`await`**](https://docs.python.org/3.6/reference/expressions.html#await) указывает, что при выполнении следующего за ним выражения возможно переключение с текущей сопрограммы на другую или на основной поток выполнения.
Соответственно выражение после await тоже не простое, это должен быть [**`awaitable`**](https://docs.python.org/3.6/glossary.html#term-awaitable) объект. Используя await в какой-либо корутине, мы таким образом объявляем, что корутина может отдавать управление обратно в event loop, который, в свою очередь, запустит какую-либо следующую задачу.

**async**

* **`async def`** — определяет native coroutine function, результатом вызова которой будет объект-сопрограмма native coroutine, пока еще не запущенная.

* **`async for`** — определяет, что итератор используемый в цикле, при получении следующего значения может переключать выполнение с текущей сопрограммы.

* **`async with`** — определяет, что при входе в контекстный блок и выходе из него может быть переключение выполнения с текущей сопрограммы.

asyncio оперирует следующими понятиями: циклы событий, корутины и футуры.

* **цикл событий (event loop)** управляет выполнением различных задач: регистрирует поступление и запускает в подходящий момент. Это бесконечный цикл, который берёт события из очереди и как-то их обрабатывает. А в некоторых промежутках — смотрит, не произошло ли каких-нибудь IO-событий и тогда добавляет в очередь событие об этом, чтобы потом обработать.
* **корутины** — специальные функции, похожие на генераторы python, от которых ожидают (await), что они будут отдавать управление обратно в цикл событий. Необходимо, чтобы они были запущены именно через цикл событий
* **футуры** — объекты, в которых хранится текущий результат выполнения какой-либо задачи. Это может быть информация о том, что задача ещё не обработана или уже полученный результат; а может быть вообще исключение

In [27]:
import asyncio
import async_timeout
from aiohttp import ClientSession

In [16]:
async def fetch_url(session, sem, url, key=None, timeout=10):
    '''
    Определяем native coroutine function.
    Args:
        param1: Объект сессии
        param2: Ограничитель в виде объекта Semaphore
        param3: URL
    Returns:
        Словарь с тремя ключами:
            letter: буква, фамилии на которую мы искали
            html: текст web-страницы
            error: сообщение об ошибке
    Raises:
        Exception если нет ответа боле 10 секунд.
    
    '''
    async with sem:
        try:
            with async_timeout.timeout(timeout):
                async with session.get(url) as response:
                    if response.status == 200:
                        html = await response.text()
                        return {
                            "url": url,
                            "html": html,
                            "error": None,
                            "key": key
                        }
                    else:
                        return {
                            "url": url,
                            "html": None,
                            "error": response.status,
                            "key": key
                        }
        except Exception as err:
            return {"url": key, "html": None, "error": err, "key": key}

In [17]:
async def get_persons(url, letters, campus_id, simultaneous_requests_num=100):
    tasks = []
    # создаём объект класса Semaphore с ограничением в 100 корутин
    sem = asyncio.Semaphore(simultaneous_requests_num)

    # Создаём клиентскую сессию, в контексте которой будут выполняться все запросы
    async with ClientSession() as session:
        for letter in letters:
            url_params = {"letter": letter, "campus_id": campus_id}
            url = url.format(**url_params)
            # Функция ensure_future принимает на вход корутину и планирует её выполнение,
            # возврящая future вида Task. Таким образом корутина помещается в цикл событий.
            # https://docs.python.org/3/library/asyncio-task.html#asyncio.Task
            task = asyncio.ensure_future(
                fetch_url(session, sem, url=url, key=letter))
            tasks.append(task)
        # собираем все futures в один awaitable список
        responses = await asyncio.gather(*tasks)
        return responses

In [18]:
# создаём цикл событий
loop = asyncio.get_event_loop()

# планируем выполнение новой корутины
uber_future = asyncio.ensure_future(
    get_persons(HSE_URL, letters=CYRILLIC_ALPHBET, campus_id=135083))
# запускаем запланированные задачи и получаем список результатов.
persons_by_letter = loop.run_until_complete(uber_future)

In [24]:
persons_by_letter[1]["key"]

'Б'